#6.6 RetrievalQA

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA


In [17]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

@traceable
def generate_response(chain, prompt):
    return chain.invoke(prompt)


In [15]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.invoke("Describe Victory Mansions")


{'query': 'Describe Victory Mansions',
 'result': 'Based on the provided text, Victory Mansions is a building with glass doors, hallways that smell of boiled cabbage and old rag mats, and a non-functioning elevator (lift) at least some of the time due to electricity cuts.  Posters depicting a large face of a man with a caption "BIG BROTHER IS WATCHING YOU" are located on each landing.  The building has at least seven flights of stairs.\n'}

In [18]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

generate_response(chain, "Describe Victory Mansions")

{'query': 'Describe Victory Mansions',
 'result': 'Victory Mansions, as depicted in the opening chapter of *Nineteen Eighty-Four*, is a decaying apartment building in Oceania, reflecting the Party\'s control and neglect.  The entrance hall is characterized by unpleasant smells of boiled cabbage and old rag mats.  A large poster of Big Brother dominates each landing, its eyes seemingly following the observer, reinforcing the constant surveillance.  The building\'s elevator is frequently out of order due to electricity cuts implemented as part of the Party\'s "economy drive," forcing residents like Winston Smith to climb numerous flights of stairs (seven in Winston\'s case).  The building\'s dilapidation and the omnipresent posters of Big Brother underscore the oppressive atmosphere and lack of privacy within Oceania, with telescreens within the apartments further emphasizing this pervasive surveillance.  The building itself is a physical manifestation of the Party\'s control over its ci

In [19]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(),
)

generate_response(chain, "Describe Victory Mansions")


{'query': 'Describe Victory Mansions',
 'result': "Victory Mansions is a building with a hallway that smells of boiled cabbage and old rag mats.  A large poster depicting a man's face (described in some excerpts as being over a meter wide, and in others as belonging to a man of about forty-five with a heavy black moustache) is tacked to the hallway wall.  The building has a lift (elevator) that is seldom working, and the electric current is cut off during daylight hours.  The flats are seven flights up, requiring the use of stairs.  Some excerpts mention glass doors.\n"}